In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('dataset_tp1.csv')

In [3]:
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [4]:
df.shape

(5171, 4)

In [5]:
df.drop_duplicates()
df.dropna()
df.shape

(5171, 4)

In [6]:
X = df['text']
Y = df['label_num']

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
Example = [
    'Ceci est le premier document.',
    'Ce document est le deuxième document.',
    'Et ceci est le troisième.',
    'Est-ce le premier document ?',
]

vector1 = CountVectorizer(ngram_range=(1, 1))
X_temp = vector1.fit_transform(Example)
bow = vector1.get_feature_names_out()
print(bow)

['ce' 'ceci' 'deuxième' 'document' 'est' 'et' 'le' 'premier' 'troisième']


In [10]:
from sklearn.feature_extraction.text import CountVectorizer


vector1 = CountVectorizer(ngram_range=(1, 2))
X1 = vector1.fit_transform(Example)
bow = vector1.get_feature_names_out()
print(bow)


['ce' 'ce document' 'ce le' 'ceci' 'ceci est' 'deuxième'
 'deuxième document' 'document' 'document est' 'est' 'est ce' 'est le'
 'et' 'et ceci' 'le' 'le deuxième' 'le premier' 'le troisième' 'premier'
 'premier document' 'troisième']


In [11]:

vector2 = CountVectorizer(ngram_range=(1, 3))
X2 = vector2.fit_transform(Example)
bow2 = vector2.get_feature_names_out()
print(bow2)

['ce' 'ce document' 'ce document est' 'ce le' 'ce le premier' 'ceci'
 'ceci est' 'ceci est le' 'deuxième' 'deuxième document' 'document'
 'document est' 'document est le' 'est' 'est ce' 'est ce le' 'est le'
 'est le deuxième' 'est le premier' 'est le troisième' 'et' 'et ceci'
 'et ceci est' 'le' 'le deuxième' 'le deuxième document' 'le premier'
 'le premier document' 'le troisième' 'premier' 'premier document'
 'troisième']


In [12]:
vector3 = CountVectorizer(analyzer='char')
X3 = vector3.fit_transform(Example)
bow3 = vector3.get_feature_names_out()
print(bow3)

[' ' '-' '.' '?' 'c' 'd' 'e' 'i' 'l' 'm' 'n' 'o' 'p' 'r' 's' 't' 'u' 'x'
 'è']


In [14]:
vector4 = CountVectorizer(analyzer='char_wb')
X4 = vector4.fit_transform(Example)
bow4 = vector4.get_feature_names_out()
print(bow4)

[' ' '-' '.' '?' 'c' 'd' 'e' 'i' 'l' 'm' 'n' 'o' 'p' 'r' 's' 't' 'u' 'x'
 'è']


In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
vector = TfidfVectorizer()
X_encoded = vector.fit_transform(X)

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.neighbors import KNeighborsClassifier

from sklearn import svm

def find_best_params(x,y):
    models={
        'SVM':{
            'model':svm.SVC(),
            'params':{
                'C':[1,10,20],
                'kernel': ['linear', 'rbf'],
            }
        },
        'Multinomial_NB':{
            'model':MultinomialNB(),
            'params':{
                'alpha':[1.0, 2.0, 5.0]
            }
        },


        'Random_Forest': {
            'model': RandomForestClassifier(),
            'params': {
                'n_estimators': [10, 20,50],
                'max_depth': [None, 10],

            }
        } ,

        'KNN': {
            'model': KNeighborsClassifier(),
            'params': {
                'n_neighbors': [3, 5, 7],
                
            }
        },
    }

    scores = []
   

    cv = ShuffleSplit(n_splits=5 , test_size=0.3 , random_state=42)
    for algo , params in models.items():
        grid_cv = GridSearchCV(params['model'],params['params'] , cv=cv , return_train_score=False )
        grid_cv.fit(x,y)
        scores.append({
            'model_name': algo,
            'best_score': grid_cv.best_score_,
            'best_params': grid_cv.best_params_

        })


    scores_df = pd.DataFrame(scores,columns=['model_name','best_score','best_params'])
    return scores_df    

In [44]:
find_best_params(X_encoded  , Y)

,model_name,best_score,best_params
0,SVM,0.987246,"{'C': 10, 'kernel': 'rbf'}"
1,Multinomial_NB,0.879227,{'alpha': 1.0}
2,Random_Forest,0.973913,"{'max_depth': None, 'n_estimators': 50}"
3,KNN,0.961739,{'n_neighbors': 3}


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
X_train, X_test, Y_train, Y_test = train_test_split(X_encoded, Y, test_size=0.25, random_state=42)
best_model = svm.SVC(kernel='linear',C=10)
best_model.fit(X_train,Y_train)
Y_pred = best_model.predict(X_test)
report = classification_report(Y_test, Y_pred)
print(report)


              precision    recall  f1-score   support

           0       0.99      0.99      0.99       930
           1       0.97      0.98      0.98       363

    accuracy                           0.99      1293
   macro avg       0.98      0.99      0.98      1293
weighted avg       0.99      0.99      0.99      1293
